In [1468]:
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import os 
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
data = pd.read_csv('Spiski2 - Студенты (1).csv')
data.head()

,ФИО,Группа,Пол (М/Ж),Место жительства\nГород,Кол-во долгов\n(Число),Кол-во пересдач,Бюджет или Договор,После: \nШколы/Вуз/СПО,Вред. привычки\n(Есть/Нет),Самообразование\n(Да/Нет),Национальность,Отчислен/Обучаюсь
0,Антипова Евгения Владимировна,АУБП-18,Ж,с.Васильевка,0,0,Бюджет,Школа,Нет,Да,Русская,Обучаюсь
1,Балашов Сергей,АУБП-18,М,Арти,0,0,Договор,Школа,Нет,Да,Русский,Обучаюсь
2,Герасимов Глеб Рафикович,АУБП-18,М,Екатеринбург,2,2,Договор,Школа,Нет,Да,Русский,Обучаюсь
3,Горбачев Константин Евгеньевич,АУБП-18,М,Екатеринбург,1,1,Договор,Школа,Нет,Да,Русский,Обучаюсь
4,Гордеев Дмитрий Владимирович,АУБП-18,М,Екатеринбург,0,3,Договор,Школа,Нет,Да,Русский,Обучаюсь


Предобработка датасета

In [1469]:
print(data['Национальность']. unique ())

['Русская' 'Русский' 'Русский ' 'Кореец' 'Еврей' 'Руссский' 'Руская']


In [1470]:
data.loc[data["Национальность"] == "Русская", "Национальность"] = 1
data.loc[data["Национальность"] == "Русский", "Национальность"] = 1
data.loc[data["Национальность"] == "Русский ", "Национальность"] = 1
data.loc[data["Национальность"] == "Руссский", "Национальность"] = 1
data.loc[data["Национальность"] == "Руская", "Национальность"] = 1
data.loc[data["Национальность"] == "Кореец", "Национальность"] = 0
data.loc[data["Национальность"] == "Еврей", "Национальность"] = 0
print(data['Национальность']. unique ())

[1 0]


In [1471]:
print(data['Отчислен/Обучаюсь']. unique ())

['Обучаюсь' 'Отчислен' 'Обучаюсь ']


In [1472]:
data.loc[data["Отчислен/Обучаюсь"] == "Обучаюсь ", "Отчислен/Обучаюсь"] = "Обучаюсь"
print(data['Отчислен/Обучаюсь']. unique ())

['Обучаюсь' 'Отчислен']


In [1473]:
print(data['Самообразование\n(Да/Нет)']. unique ())

['Да' 'Нет']


In [1474]:
data.loc[data["Самообразование\n(Да/Нет)"] == "Да", "Самообразование\n(Да/Нет)"] = 1
data.loc[data["Самообразование\n(Да/Нет)"] == "Нет", "Самообразование\n(Да/Нет)"] = 0
data.rename(columns = {'Самообразование\n(Да/Нет)': 'Самообразование'}, inplace = True)

In [1475]:
print(data['Вред. привычки\n(Есть/Нет)']. unique ())

['Нет' 'Да']


In [1476]:
data.loc[data["Вред. привычки\n(Есть/Нет)"] == "Да", "Вред. привычки\n(Есть/Нет)"] = 1
data.loc[data["Вред. привычки\n(Есть/Нет)"] == "Нет", "Вред. привычки\n(Есть/Нет)"] = 0
data.rename(columns = {'Вред. привычки\n(Есть/Нет)': 'Вред. привычки'}, inplace = True)

In [1477]:
print(data['После: \nШколы/Вуз/СПО']. unique ())

['Школа' 'Колледж' 'Школа ']


In [1478]:
data.loc[data["После: \nШколы/Вуз/СПО"] == "Школа ", "После: \nШколы/Вуз/СПО"] = "Школа"

In [1479]:
print(data['Бюджет или Договор']. unique ())

['Бюджет ' 'Договор' 'Бюджет']


In [1480]:
data.loc[data["Бюджет или Договор"] == "Бюджет ", "Бюджет или Договор"] = "Бюджет"
data.loc[data["Бюджет или Договор"] == "Бюджет", "Бюджет или Договор"] = 1
data.loc[data["Бюджет или Договор"] == "Договор", "Бюджет или Договор"] = 0
data.rename(columns = {'Бюджет или Договор': 'Бюджет'}, inplace = True)

In [1481]:
print(data['Место жительства\nГород'].unique ())

['с.Васильевка' 'Арти' 'Екатеринбург' 'Камышлов' 'Екатеринбург '
 'Магнитогорск' 'Рефтинский' 'Снежинск' 'Советский' 'Асбест' 'Мортка'
 'Полевской' 'Михайловск' 'Первоуральск' 'Ирбит' 'Березовский' 'Рубцовск'
 'Реж' 'Верхний Уфалей' 'Гай' 'Апатиты' 'Нефтеюганск' 'Екатеринбруг'
 'Нижний тагил' 'Лесной' 'Уренгой']


In [1482]:
data.loc[data["Место жительства\nГород"] == "Екатеринбург ", "Место жительства\nГород"] = 1
data.loc[data["Место жительства\nГород"] == "Екатеринбург", "Место жительства\nГород"] = 1
data.loc[data["Место жительства\nГород"] == "Екатеринбруг", "Место жительства\nГород"] = 1
data.loc[data["Место жительства\nГород"] != 1, "Место жительства\nГород"] = 0

In [1483]:
data.rename(columns = {'Место жительства\nГород': 'Екатеринбург'}, inplace = True)
print(data['Екатеринбург'].unique ())

[0 1]


In [1484]:
data.rename(columns = {'Кол-во долгов\n(Число)': 'Кол-во долгов'}, inplace = True)

In [1485]:
print(data['После: \nШколы/Вуз/СПО'].unique ())

['Школа' 'Колледж']


In [1486]:
data = data.assign(Женщина = data['Пол (М/Ж)']=='Ж').round(2)
data = data.assign(Мужчина = data['Пол (М/Ж)']=='М').round(2)

In [1487]:
data = data.assign(Школа = data['После: \nШколы/Вуз/СПО']=='Школа').round(2)
data = data.assign(Вуз = data['После: \nШколы/Вуз/СПО']=='Вуз').round(2)
data = data.assign(СПО = data['После: \nШколы/Вуз/СПО']=='Колледж').round(2)

In [1488]:
data = data.astype({'Женщина': 'int', 'Мужчина' : 'int', 'Школа' : 'int', 'Вуз' : 'int', 'СПО' : 'int', 'Екатеринбург' : 'int'})
data = data.astype({'Вред. привычки': 'int', 'Самообразование' : 'int', 'Национальность': 'int','Бюджет': 'int'})

In [1489]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ФИО                    94 non-null     object
 1   Группа                 94 non-null     object
 2   Пол (М/Ж)              94 non-null     object
 3   Екатеринбург           94 non-null     int32 
 4   Кол-во долгов          94 non-null     int64 
 5   Кол-во пересдач        94 non-null     int64 
 6   Бюджет                 94 non-null     int32 
 7   После: 
Школы/Вуз/СПО  94 non-null     object
 8   Вред. привычки         94 non-null     int32 
 9   Самообразование        94 non-null     int32 
 10  Национальность         94 non-null     int32 
 11  Отчислен/Обучаюсь      94 non-null     object
 12  Женщина                94 non-null     int32 
 13  Мужчина                94 non-null     int32 
 14  Школа                  94 non-null     int32 
 15  Вуз                    94

In [1490]:
data.head()

,ФИО,Группа,Пол (М/Ж),Екатеринбург,Кол-во долгов,Кол-во пересдач,Бюджет,После: \nШколы/Вуз/СПО,Вред. привычки,Самообразование,Национальность,Отчислен/Обучаюсь,Женщина,Мужчина,Школа,Вуз,СПО
0,Антипова Евгения Владимировна,АУБП-18,Ж,0,0,0,1,Школа,0,1,1,Обучаюсь,1,0,1,0,0
1,Балашов Сергей,АУБП-18,М,0,0,0,0,Школа,0,1,1,Обучаюсь,0,1,1,0,0
2,Герасимов Глеб Рафикович,АУБП-18,М,1,2,2,0,Школа,0,1,1,Обучаюсь,0,1,1,0,0
3,Горбачев Константин Евгеньевич,АУБП-18,М,1,1,1,0,Школа,0,1,1,Обучаюсь,0,1,1,0,0
4,Гордеев Дмитрий Владимирович,АУБП-18,М,1,0,3,0,Школа,0,1,1,Обучаюсь,0,1,1,0,0


In [1491]:
data.describe()

,Екатеринбург,Кол-во долгов,Кол-во пересдач,Бюджет,Вред. привычки,Самообразование,Национальность,Женщина,Мужчина,Школа,Вуз,СПО
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.00000,94.0,94.00000
mean,0.670213,1.042553,1.276596,0.414894,0.319149,0.882979,0.978723,0.255319,0.744681,0.93617,0.0,0.06383
std,0.472657,1.625832,1.931094,0.495346,0.468646,0.323169,0.145079,0.438378,0.438378,0.24576,0.0,0.24576
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.250000,1.00000,0.0,0.00000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.00000,0.0,0.00000
75%,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.750000,1.000000,1.00000,0.0,0.00000
max,1.000000,6.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.0,1.00000


In [1492]:
data.drop(data.columns [[0,1,2,7]], axis= 1 , inplace= True)
data.isna()

,Екатеринбург,Кол-во долгов,Кол-во пересдач,Бюджет,Вред. привычки,Самообразование,Национальность,Отчислен/Обучаюсь,Женщина,Мужчина,Школа,Вуз,СПО
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,False,False,False,False,False,False,False,False,False,False,False,False,False
90,False,False,False,False,False,False,False,False,False,False,False,False,False
91,False,False,False,False,False,False,False,False,False,False,False,False,False
92,False,False,False,False,False,False,False,False,False,False,False,False,False


Создание тренировочного и тестового набора

In [1493]:
train = data.sample (frac= 0.7 ,random_state= 58 )
test = data.drop (train. index )

In [1494]:
print(train.head())

    Екатеринбург  Кол-во долгов  Кол-во пересдач  Бюджет  Вред. привычки  \
56             1              0                2       1               0   
26             1              1                2       0               0   
89             1              0                0       0               0   
31             1              0                0       1               0   
77             1              1                0       0               1   

    Самообразование  Национальность Отчислен/Обучаюсь  Женщина  Мужчина  \
56                1               1          Обучаюсь        1        0   
26                1               1          Обучаюсь        0        1   
89                1               1          Обучаюсь        0        1   
31                1               1          Обучаюсь        1        0   
77                1               1          Обучаюсь        0        1   

    Школа  Вуз  СПО  
56      1    0    0  
26      1    0    0  
89      0    0    1  
31  

In [1495]:
print(test.head(28))

    Екатеринбург  Кол-во долгов  Кол-во пересдач  Бюджет  Вред. привычки  \
0              0              0                0       1               0   
3              1              1                1       0               0   
12             1              4                3       0               0   
14             1              0                0       1               0   
21             1              0                1       0               0   
23             1              0                0       0               0   
25             0              4                3       0               1   
29             0              0                0       0               0   
32             0              0                1       0               0   
33             1              4                2       0               1   
34             1              6                0       0               1   
35             1              5                0       1               0   
37          

In [1496]:
print(train. shape , test. shape )

(66, 13) (28, 13)


In [1497]:
X_train = train.drop('Отчислен/Обучаюсь', axis = 1)
Y_train = train['Отчислен/Обучаюсь']

In [1498]:
X_test = test.drop('Отчислен/Обучаюсь', axis = 1)
Y_test = test['Отчислен/Обучаюсь']

In [1499]:
#Метод k-ближайших соседей (K-Nearest Neighbors)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [1500]:
knn.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

In [1501]:
print(knn.score(X_train, Y_train))
print(knn.score(X_test, Y_test))

0.9696969696969697
0.8928571428571429


In [1502]:
pred_y = knn.predict(X_test)
from sklearn import metrics
print(accuracy_score(pred_y, Y_test))
print(confusion_matrix(pred_y,Y_test))
print(metrics.classification_report(pred_y,Y_test))

0.8928571428571429
[[25  3]
 [ 0  0]]
              precision    recall  f1-score   support

    Обучаюсь       1.00      0.89      0.94        28
    Отчислен       0.00      0.00      0.00         0

    accuracy                           0.89        28
   macro avg       0.50      0.45      0.47        28
weighted avg       1.00      0.89      0.94        28



C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1503]:
#логическая регрессия
from sklearn.linear_model import LogisticRegression 
LR = LogisticRegression(random_state=0)
LR.fit(X_train, Y_train) 

LogisticRegression(random_state=0)

In [1504]:
print(LR.score(X_train, Y_train))
print(LR.score(X_test, Y_test))

0.9696969696969697
0.9285714285714286


In [1505]:
pred_y = LR.predict(X_test)
from sklearn import metrics
print(accuracy_score(pred_y, Y_test))
print(confusion_matrix(pred_y,Y_test))
print(metrics.classification_report(pred_y,Y_test))

0.9285714285714286
[[25  2]
 [ 0  1]]
              precision    recall  f1-score   support

    Обучаюсь       1.00      0.93      0.96        27
    Отчислен       0.33      1.00      0.50         1

    accuracy                           0.93        28
   macro avg       0.67      0.96      0.73        28
weighted avg       0.98      0.93      0.95        28



In [1506]:
#метод опорных векторов
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, Y_train)

SVC()

In [1507]:
print(clf.score(X_train, Y_train))
print(clf.score(X_test, Y_test))

0.9696969696969697
0.8928571428571429


In [1508]:
pred_y = clf.predict(X_test)
from sklearn import metrics
print(accuracy_score(pred_y, Y_test))
print(confusion_matrix(pred_y,Y_test))
print(metrics.classification_report(pred_y,Y_test))

0.8928571428571429
[[25  3]
 [ 0  0]]
              precision    recall  f1-score   support

    Обучаюсь       1.00      0.89      0.94        28
    Отчислен       0.00      0.00      0.00         0

    accuracy                           0.89        28
   macro avg       0.50      0.45      0.47        28
weighted avg       1.00      0.89      0.94        28



C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1509]:
#случайный лес
from sklearn.ensemble import RandomForestClassifier
clas = RandomForestClassifier(max_depth=15,random_state=0)
clas.fit(X_train, Y_train)

RandomForestClassifier(max_depth=15, random_state=0)

In [1510]:
print(clas.feature_importances_)

[6.48813042e-02 3.86637145e-01 2.64328224e-01 3.56008795e-02
 5.61779056e-02 2.06379309e-02 5.49138362e-05 1.09843979e-01
 5.81688286e-02 3.84767956e-04 0.00000000e+00 3.28412146e-03]


In [1511]:
print(clas.score(X_train, Y_train))
print(clas.score(X_test, Y_test))

1.0
0.8928571428571429


In [1512]:
pred_y = clas.predict(X_test)
from sklearn import metrics
print(accuracy_score(pred_y, Y_test))
print(confusion_matrix(pred_y,Y_test))
print(metrics.classification_report(pred_y,Y_test))

0.8928571428571429
[[25  3]
 [ 0  0]]
              precision    recall  f1-score   support

    Обучаюсь       1.00      0.89      0.94        28
    Отчислен       0.00      0.00      0.00         0

    accuracy                           0.89        28
   macro avg       0.50      0.45      0.47        28
weighted avg       1.00      0.89      0.94        28



C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1513]:
best_RF = None
best_accuracy_RF = 0
best_est_RF = 0
best_depth_RF = 0
for est in tqdm(range(2,51)):
    for depth in range(2,25):
        clas = RandomForestClassifier(random_state = 47 ,n_estimators = est, max_depth = depth)
        clas.fit(X_train, Y_train)
        prediction_valid_RF = clas.predict(X_test)
        accuracy_RF = accuracy_score(pred_y,Y_test)
        if best_accuracy_RF < accuracy_RF:
            best_RF = RF
            best_depth_RF = depth
            best_est_RF = est
            best_accuracy_RF = accuracy_RF

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [00:41<00:00,  1.19it/s]


In [1514]:
#Деревья решений
from sklearn.tree import DecisionTreeClassifier
best_DT = None
best_accuracy_DT = 0
best_depth_DT = 0
for depth in tqdm(range(2,50)):
    DT = DecisionTreeClassifier(random_state = 88, max_depth = depth)
    DT.fit(X_train, Y_train)
    prediction_valid_DT = DT.predict(X_test)
    accuracy_DT = accuracy_score(pred_y,Y_test)
    if best_accuracy_DT < accuracy_DT:
        best_DT = DT
        best_accuracy_DT = accuracy_DT
        best_depth_DT = depth

100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 257.98it/s]


In [1515]:
pred_y = DT.predict(X_test)
print(accuracy_score(pred_y, Y_test))
print(confusion_matrix(pred_y,Y_test))
print(metrics.classification_report(pred_y,Y_test))

0.9285714285714286
[[24  1]
 [ 1  2]]
              precision    recall  f1-score   support

    Обучаюсь       0.96      0.96      0.96        25
    Отчислен       0.67      0.67      0.67         3

    accuracy                           0.93        28
   macro avg       0.81      0.81      0.81        28
weighted avg       0.93      0.93      0.93        28



In [1516]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
dtc_model = GridSearchCV(DecisionTreeClassifier(), pred_y, cv=10)
plt.figure(figsize=(15,15))
plot_tree(dtc_model.best_estimator_, filled=True, impurity=True) 

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

<Figure size 1500x1500 with 0 Axes>